In [ ]:
##############      Configuración      ##############
from dotenv import dotenv_values
from plantilla import plantilla

env = dotenv_values("globals.env")

proyecto = 'Códigos'
subproyecto = 'Testing - Deep Learning con MapTilesDownloader'

globales = plantilla(proyecto=proyecto, 
                     subproyecto=subproyecto,
                     path_proyectos=env['PATH_PROYECTOS']
           )

path_proyecto   = globales[0]     # Ubicación de la carpeta del Proyecto
path_datain     = globales[1]
path_dataout    = globales[2]     # Bases procesadas por tus scripts
path_scripts    = globales[3]
path_figures    = globales[4]     # Output para las figuras/gráficos
path_maps       = globales[5]     # Output para los mapas (html o imagen)
path_tables     = globales[6]     # Output para las tablas (imagen o excel)
path_programas  = globales[7]

In [9]:
import numpy as np
import geopandas as gpd
icpag = gpd.read_file(r"R:\Shapefiles\ICPAG\Sin barrios pop y cerr\Aglomerados de mas de 500k habitantes\base_icpag_500k.shp")
icpag = icpag.to_crs(epsg=3857)

In [ ]:
import cartopy.io.img_tiles as cimgt

icpag = icpag.reset_index(drop=True)
icpag_amba = icpag[icpag.AMBA_legal == 1]
icpag_amba = icpag_amba[icpag_amba.area <= 1_000_000] # Aprox p90
icpag_amba = icpag_amba.reset_index(drop=True)

map_provider = cimgt.GoogleTiles(style='satellite')

In [ ]:
icpag_amba

In [ ]:
import gc
from shapely.geometry import box

import cartopy.crs as crs
import cartopy.io.img_tiles as cimgt
import cartopy.feature as cfeature

import seaborn as sns; sns.set(style="ticks", color_codes=True)
import matplotlib.pyplot as plt
import matplotlib.colors as colors 
from matplotlib.patches import Patch
from matplotlib_scalebar.scalebar import ScaleBar

from PIL import Image

icpag.iloc[0:1,:]

def create_map_from_geometry(
    icpag, index, 
    map_provider=map_provider, zoom=18, map_name='ESRI_WI',  
    path_figures=path_figures):

    Image.MAX_IMAGE_PIXELS = 2160000 
    # Genero la máscara para el gráfico y obtengo el extent
    mask_inv = icpag.iloc[index:index+1,:]
    link = mask_inv.at[index,'link']
    bbox = mask_inv.total_bounds
    geom =box(*bbox)
    mask = mask_inv.copy()
    mask['geometry'] = geom

    # Gráfico
    fig, ax = plt.subplots(subplot_kw={'projection': crs.epsg(3857)}, dpi=300, facecolor = 'black')

    # Limita la visualización a un área determinada
    ax.set_extent(
        [bbox[0], bbox[2], bbox[1], bbox[3]],
        crs=crs.epsg(3857)
    )

    # Agrego mapa de fondo
    ax.add_image(map_provider, zoom)

    # # Quito bordes y grilla
    ax.spines['geo'].set_edgecolor('black')

    # # Añade la máscara
    mask.difference(mask_inv).plot(ax=ax, facecolor='black', edgecolor='black', linewidth=0.0) 
    plt.tight_layout()
    
    fig.savefig(fr"{path_figures}\{map_name}_{link}_{index}.tiff")
    plt.close()
    gc.collect()

In [ ]:
import pandas as pd
icpag_amba_a = icpag_amba.set_index('link').loc['064277201':'064414308',:]
icpag_amba_b = icpag_amba.set_index('link').loc['067562706':,:]
icpag_amba = pd.concat([icpag_amba_a,icpag_amba_b]).reset_index()

list = np.array_split(range(icpag_amba.shape[0]), 2)

In [ ]:
from tqdm import tqdm
for index in tqdm(list[0]):
    create_map_from_geometry(icpag_amba, index, map_name='Google_maps')

In [ ]:
import os
dir = r'R:\Tesis Nico\Códigos\outputs\figures\Testing - Deep Learning con MapTilesDownloader'
os.listdir(dir)

In [ ]:
import re
comp = re.compile('_\d+.tiff')
re.sub(comp,'.tiff', 'ESRI_WI_020010101_0.tiff')

In [8]:
import os
import re
from tqdm import tqdm
# [os.rename(f, f.replace('_', '-')) for f in os.listdir('.') if not f.startswith('.')]
comp = re.compile('_\d+.tiff')

for f in tqdm(os.listdir(dir)[10:]):
    if not f.startswith(dir):
        new_name = re.sub(comp,'.tiff', f'{dir}\\{f}')
        if not os.path.exists(new_name):
            os.rename(f'{dir}\\{f}', new_name)


100%|██████████| 28153/28153 [04:27<00:00, 105.23it/s]


In [ ]:
f =  dir + '\\' + 'ESRI_WI_020010101_0.tiff'
os.rename(f, re.sub(comp,'.tiff', f))

In [ ]:
os.listdir(dir)[:10]